# Convolutional Neural Network

### Importing the libraries

In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator # 이미지 전처리에 필요

In [ ]:
tf.__version__

'2.15.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [ ]:
# 이미지 전처리
# 1. 훈련세트에 있는 모든 이미지에 변환 적용 - 과적합 방지. 이 과정은 안하면 훈련셋(정확도 매우 큼.. 과적합)과 테스트셋의 정확도 차가 큼
# 컴퓨터 시각에서는 과적합 피하는 방법이 변환 적용 하는 것.
# 여기서의 변환: 이미지를 단순히 기하학적으로 변환 or 확대 or 회전하는 것들을 의미
# 기본적으로 픽셀의 일부를 바꿔주는 기하학적인 변환을 적용 or 이미지 회전 or 가로로 뒤집기 or 확대/축소도 함. = 이미지 증대.
# 이미지 증대(image augmentation) - 훈련세트에 있는 이미지들을 변환해서 CNN모델이 존재하는 이미지들을 과하게 학습하지 않도록하는 것으로 구성됨.
# 이 변환들을 적용하면 새로운 이미지가 생김.  = 이미지 증대(훈련세트 이미지들의 다양성을 증대)
train_datagen = ImageDataGenerator(  # 훈련 세트의 이미지에 변환을 적용해줄 모든 도구를 나타냄.
        rescale=1./255, # feature scaling(신경망 훈련 시 필수). 값을 255로 나눠서 모든 픽셀 하나하나에 특징 스케일링을 적용.(각 픽셀 값이 0 ~ 255 -> 255로 나누면 0~1사이 값이됨)
        shear_range=0.2, # 나머지는 훈련 세트의 이미지들에 이미지 증대를 수행할 변환들(과적합 예방).
        zoom_range=0.2,
        horizontal_flip=True)
# 디렉토리에서 훈련 세트에 접근해서 그걸 가져오고 동시에 이미지 배치를 만들고
# 머신의 계산이 덜 힘들도록 계산을 줄이기 위해 크기를 재설정해야 한다면 그렇게 해줌. 결국에는 더 작은 크기가 더 놀라운 결과를 가져다주기 때문.
training_set = train_datagen.flow_from_directory( # train_datagen이라는 이미지 증대 도구를 훈련 세트의 이미지에 연결하는 것.
        'dataset/training_set',
        target_size = (64, 64), # 이미지의 최종 크기(컨볼루션 신경망에 입력될 이미지들). (150, 150)으로 해봤더니 훈련이 너무 오래 걸림
        batch_size = 32, # 각 배치에 들어가는 이미지의 개수. 32가 전형적인 기본값
        class_mode = 'binary') # binary 또는 categorical 중 하나. 고양이 or 강이지이므로 이진결과.


### Preprocessing the Test set

In [ ]:
# 테스트 이미지에 변환을 적용
# 훈련용 이미지와 똑같은 변환은 적용 안함. 새로운 이미지나 마찬가지이므로 건드리지 않음.
# 한 가지 해줄 것은 픽셀을 규모를 바꾸는 것. 피처 스케일링을 하는 이유는 CNN의 predict 메서드가 훈련 세트에 적용한 것과 같은 스케일링에 적용되어야 하기 때문.
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(
        'dataset/test_set',
        target_size = (64, 64),
        batch_size = 32,
        class_mode = 'binary')


## Part 2 - Building the CNN

### Initialising the CNN

In [ ]:
cnn = tf.keras.models.Sequential() # Sequential 클래스 -> 인공신경망을 일련의 계층들로 생성할 수 있게 해줌.

### Step 1 - Convolution

In [ ]:
# 컨볼루션 계층. 컨볼루션 계층은 특정 클래스인 Conv2D 클래스의 객체가 됨.
# Conv2D 클래스는 완전 연결 계층을 구축하게 해주는 Dense클래스와 같은 모듈에 속함.
# 이 클래스에는 4개의 중요한 파라미터가 들어감. 나머지는 기본값.
# filter -> 이미지에 적용하고 싶은 특징 감지기(feature detector)의 수. 필터나 커널이라고도 부름.
# filter를 다른 값 넣어도 상관없음. Classic한 아키텍처는 첫번째 컨볼루션 계층과 두번째 컨볼루션 계층에 32 필터를 가진 것으로 구성.
# kernal_size -> 특징 감지기의 크기(행렬) 3을 선택하면 크기 = 3X3. 여기서는 3차원을 원하므로 3을 작성.
# activation -> ReLU(정류 활성 함수) 파라미터 선택
# input_shape -> 맨 첫번째 layer을 추가하게 되면 이게 컨볼루션 layer이든 dense layer이든 입력에 input_shape를 명시해줘야 함.
# 여기서는 컬러 이미지로 작업하므로 3차원이되고 이게 RGB 색상에 상응함.
# 위에서 이미지를 64X64로 줄였으므로 (64. 64. 3)이 됨. 흑백이미지였다면 (64. 64. 1)

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

In [ ]:
# 이게 CNN에 컨볼루션 계층(layer)을 추가하고 일련의 계층으로 초기화 한 것.

### Step 2 - Pooling

In [ ]:
# 여기서는 최대 풀링 적용. 컨볼루션 계층에 풀링 계층을 추가.
# MaxPool2D 클래스 이용.
# 2개의 필수 인자를 넣어줌(pool_size, strides).
# 이전 컨볼루션의 결과로 특징 맵이 생겼고, 풀링된 특징 맵을 얻기 위해 최대풀링을 사용.
# 그렇게 하면 작은 프레임이 생기는데 그 안의 칸들 중 최대 픽샐을 구하는 것.
# pool_size 파라미터에 넣을 것이 그 프레임의 크기. 사각형이므로 가로나 세로만 명시해주면 됨.
# 최대 풀링을 적용하려면 크기를 2 정도로 권장.(2X2프레임)
# strides -> 프레임이 몇개의 픽셀을 건너 뛰어서 다음 칸으로 갈것인지 결정.
# 권장되는 슬라이드는 2. 옆으로 2픽셀 씩 옮겨짐. 옮기다가 특징 맵의 경계에 도달하면 빈칸들이 생기는데
# 이것은 파라미터 padding을 이용하여 두 가지 다른 방법을 선택할 수 있음. 기본값 = valid, 다른값은 same
# valid -> 빈칸 무시. same -> 0인척하는 가짜 픽셀 추가.
# 패딩에 관해서는 너무 걱정X. 여기서는 둘다 해봤는데 최종결과는 바뀌지 않았음. 그냥 기본값(valid)으로 결정.

In [ ]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [ ]:
# input_shape파라미터는 맨 첫 번째 계층을 추가할 때만 넣어주는 것이므로 삭제
# input_shape파라미터는 첫 번째 계층을 입력 계층에 연결하려는 목적임(자동으로 입력 계층에 추가해줌).
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [ ]:
# 컨볼루션과 풀링을 1차원 벡터로 플래트닝 해줌.
# 완전 연결 신경망의 입력이 될 것임.
# 플레트닝 계층을 생성하는 방식은 특정 클래스(Flatten)의 인스턴스를 생성하는 것이므로 add메서드를 사용.
# 추가할 파라미터 없음.
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [ ]:
# 완전 연결 신경망의 입력이 될 1차원 벡터인 새 완전 연결 계층 추가.
# units -> 히든 뉴런 수.
# 더 복잡한 문제를 다루고 있으므로 컴퓨터 시각은 전처럼 클래식 데이터 세트를 데이터 마이닝하는 것보다 더 복잡함.
# 따라서 히든 뉴런 수는 큰 수를 입력할 것임.(units=128)
# activation은 최종 출력 계층에 도달하지 않았다면 rectifier activation function(정류 활성화 함수)을 사용.
# 이 과정으로 완전 연결 계층이 추가됨.
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [ ]:
# 최종 출력 layer를 추가. 이전 히든 layer와 완전 연결이 되야 함.
# 이진 분류이므로 units=1, 0인지 1인지, 강아지인지 고양이인지 나타내는 binary클래스를 인코딩하려면
# 뉴런은 하나만 있으면 됨.
# 이진분류이므로 시그모이드 함수 사용.
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [ ]:
# 이진 분류를 하고 있으므로 저번에 했던 ANN을 컴파일했던 방식과 동일.
# 옵티마이저 - 확률적 경사하강법, 손실함수 - 이진 교차 엔트로피 손실 함수, 메트릭 - accuracy 메트릭(분류모델의 수행 능력을 평가하는 최적의 방법)
# CNN을 훈련 세트에서 훈련하면서 동시에 테스트 세트에서 평가를 하려고 하면 이전(ANN)과 같지는 않지만 매우 비슷함.
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [ ]:
# training_set -> 이미지 증대를 수행하도록 ImageDataGenerator를 적용해준 training set
# 훈련 세트로 CNN을 훈련시키는 것뿐만 아니라 동시에 테스트 세트로 평가도 할 것임.
# validation_data 파라미터로 이것을 명시. CNN을 평가하려고 하는 세트임.
# test_set -> 아무런 변환도 적용하지 않았고 피처 스케일링만 적용한 테스트 세트
# 에폭 수를 10부터 하니까 정확도가 크게 수렴하지 않았음. 고로 15부터 시도. 왜냐면 1 epoch가 꽤 느림(ANN보다 훨씬 김).
# 그러다가 25로 시도 했더니 완벽. 꽤 수렴된 정확도가 나왔음(훈련셋 뿐만아니라 테스트셋도).
# 시간이 된다면 더 크게 해도 됨. 25로 하면 10분에서 15분정도 걸릴 듯.
cnn.fit(x = training_set, validation_data = test_set, epochs=25)

## Part 4 - Making a single prediction

In [ ]:
# 단일 예측. 단일 이미지를 불러와서 이 모델을 배치한 후 그 이미지가 강아지인지 고양이인지 예측하게 할 것임.
import numpy as np
from keras.preprocessing import image
test_image = image.load_img(path='dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64)) # 실제 환경에서 우리 모델로 테스트 하고 싶은 이미지를 의미
# 이미지가 predict메서드의 입력이 될 것이므로 훈련에 사용했던 것과 반드시 같은 크기여야 함. target_size = (64, 64)
# 하지만 predict 메서드가 이것을 받아들이려면 특정 형식이 필요함.
# PIL 형식을 이미지를 배열로 놓는 형식으로 변환하면 됨. predict메서드는 입력으로 배열을 예상.img_to_array()
# img_to_array() -> PIL이미지 인스턴스를 넘파이 배열 형식으로 변환.
test_image = image.img_to_array(test_image)
# predict 메서드는 훈련에 사용했던 것과 완전히 똑같은 형식에서 불러와야 함.
# 파트1 데이터 전처리 부분에서 훈련세트와 테스트 세트 전처리 시 이미지의 배치를 생성했음.
# 그러므로 CNN이 단일 이미지에서 훈련된 것이 아님. 이미지의 배치로 훈련했음.
# 고로 이 배치에 상응하는 추가적인 차원이 필요함. 배치 1에는 32장의 이미지, 배치2에는 또다른 32장의 이미지...
# 우리가 모델을 단일 이미지에 배치 한다고 해도 그 단일 이미지는 배치 안에 있어야 함(배치 안에 이미지가 한 장만 있다고 하더라도).
# 배치 안에 넣어서 CNN모델의 predict 메서드가 이 배치의 추가 차원으로 인식할 수 있게 해야 함.
# 이제 할일은 배치에 상응하고, 이미지들을 배치에 넣을 추가 차원을 더하는 것.
test_image = np.expand_dims(test_image, axis = 0) # 어디에 추가 차원을 넣어줄지에 대한 인자. 그 배치의 차원은 항상 첫번째 차원이 되야 함. axis = 0
                                          # 이미지 배치를 첫번째로 주고 각 배치 안에 다른 이미지들이 들어가기 때문.
result = cnn.predict(test_image/255.0) # test_image와 함께 불러지는 CNN의 예측이 됨. 아직은 0과 1만 반환 그게 뭔지 각각 인코딩 해줘야 함(0: 고양이, 1: 강아지).

# 0이 강아지/고양이인지 1이 강아지/고양이인지 알아내는 법은 훈련 세트 객체에서 클래스를 불러오고 인덱스 속성을 가져오는 것.
# 그러나 지금 이 코드를 코랩에서 실행할 수 없으므로 어떤 인덱스가 어떤 클래스에 상응하는지 알 수 있는 코드를 써줄 것임.
training_set.class_indices # 이것을 출력하면 클래스 인덱스를 갖게 됨. 이것으로 강아지가 1에 상응하고 고양이가 0에 상응한다는 것을 알 수 있음.

if result[0][0] > 0.5: # 배치에 접근 후 배치 안에서 배치의 첫 번째이자 유일한 요소에 접근(배치 내 단일요소에 접근). cat_or_dog_1 이미지에 대한 예측에 상응. 하나의 이미지를 다루므로 하나의 예측이 나옴.
    prediction = 'dog'
else:
    prediction = 'cat'


In [ ]:
print(prediction)

In [ ]:
# 실행은 주피터 노트북에서 해야함. 데이터 세트가 너무 크기 때문.